# Model with Skills-Have Only

## Import Libraries

In [1]:
import pandas
from sklearn.cluster import KMeans
from sklearn.neighbors import KNeighborsClassifier
import numpy as np



## Transform Employees Datasets

In [2]:
# Read source dataset
df = pandas.read_csv(filepath_or_buffer="data/survey_results_public_1.csv", sep=",", encoding="latin1")
df

,ResponseId,MainBranch,Employment,RemoteWork,CodingActivities,EdLevel,LearnCode,LearnCodeOnline,LearnCodeCoursesCert,YearsCode,...,TimeSearching,TimeAnswering,Onboarding,ProfessionalTech,TrueFalse_1,TrueFalse_2,TrueFalse_3,SurveyLength,SurveyEase,ConvertedCompYearly
0,1,None of these,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,I am a developer by profession,"Employed, full-time",Fully remote,Hobby;Contribute to open-source projects,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Too long,Difficult,NaN
2,3,"I am not primarily a developer, but I write co...","Employed, full-time","Hybrid (some remote, some in-person)",Hobby,"Masterâs degree (M.A., M.S., M.Eng., MBA, etc.)",Books / Physical media;Friend or family member...,Technical documentation;Blogs;Programming Game...,NaN,14,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Appropriate in length,Neither easy nor difficult,40205.0
3,4,I am a developer by profession,"Employed, full-time",Fully remote,I donât code outside of work,"Bachelorâs degree (B.A., B.S., B.Eng., etc.)","Books / Physical media;School (i.e., Universit...",NaN,NaN,20,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Appropriate in length,Easy,215232.0
4,5,I am a developer by profession,"Employed, full-time","Hybrid (some remote, some in-person)",Hobby,"Bachelorâs degree (B.A., B.S., B.Eng., etc.)","Other online resources (e.g., videos, blogs, f...",Technical documentation;Blogs;Stack Overflow;O...,NaN,8,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Too long,Easy,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36629,36630,I code primarily as a hobby,"Student, full-time;Not employed, and not looki...",NaN,NaN,"Secondary school (e.g. American high school, G...",Books / Physical media;Other online resources ...,Technical documentation;Blogs;Written Tutorial...,NaN,16,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Appropriate in length,Easy,NaN
36630,36631,I am a developer by profession,"Employed, full-time",Full in-person,Hobby,"Bachelorâs degree (B.A., B.S., B.Eng., etc.)","School (i.e., University, College, etc)",NaN,NaN,6,...,15-30 minutes a day,15-30 minutes a day,Just right,Innersource initiative;DevOps function;Microse...,No,Yes,Yes,Appropriate in length,Neither easy nor difficult,301536.0
36631,36632,I am a developer by profession,"Student, full-time;Employed, part-time","Hybrid (some remote, some in-person)",School or academic work,"Bachelorâs degree (B.A., B.S., B.Eng., etc.)","Other online resources (e.g., videos, blogs, f...",Technical documentation;Blogs;Stack Overflow;O...,NaN,4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Appropriate in length,Easy,NaN
36632,36633,I am a developer by profession,"Independent contractor, freelancer, or self-em...",Fully remote,Hobby;Bootstrapping a business;Freelance/contr...,"Professional degree (JD, MD, etc.)",Books / Physical media;Other online resources ...,Technical documentation;Blogs;Written Tutorial...,Udemy,19,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Too long,Neither easy nor difficult,140772.0


In [3]:
# Select only required columns
df = df[["ResponseId","YearsCodePro","LanguageHaveWorkedWith","LanguageWantToWorkWith","DatabaseHaveWorkedWith","DatabaseWantToWorkWith","PlatformHaveWorkedWith","PlatformWantToWorkWith","WebframeHaveWorkedWith","WebframeWantToWorkWith","MiscTechHaveWorkedWith","MiscTechWantToWorkWith","ToolsTechHaveWorkedWith","ToolsTechWantToWorkWith"]]
df = df.fillna('')
df


,ResponseId,YearsCodePro,LanguageHaveWorkedWith,LanguageWantToWorkWith,DatabaseHaveWorkedWith,DatabaseWantToWorkWith,PlatformHaveWorkedWith,PlatformWantToWorkWith,WebframeHaveWorkedWith,WebframeWantToWorkWith,MiscTechHaveWorkedWith,MiscTechWantToWorkWith,ToolsTechHaveWorkedWith,ToolsTechWantToWorkWith
0,1,,,,,,,,,,,,,
1,2,,JavaScript;TypeScript,Rust;TypeScript,,,,,,,,,,
2,3,5,C#;C++;HTML/CSS;JavaScript;Python,C#;C++;HTML/CSS;JavaScript;TypeScript,Microsoft SQL Server,Microsoft SQL Server,,,Angular.js,Angular;Angular.js,Pandas,.NET,,
3,4,17,C#;JavaScript;SQL;TypeScript,C#;SQL;TypeScript,Microsoft SQL Server,Microsoft SQL Server,,,ASP.NET;ASP.NET Core,ASP.NET;ASP.NET Core,.NET,.NET,,
4,5,3,C#;HTML/CSS;JavaScript;SQL;Swift;TypeScript,C#;Elixir;F#;Go;JavaScript;Rust;TypeScript,Cloud Firestore;Elasticsearch;Microsoft SQL Se...,Cloud Firestore;Elasticsearch;Firebase Realtim...,Firebase;Microsoft Azure,Firebase;Microsoft Azure,Angular;ASP.NET;ASP.NET Core ;jQuery;Node.js,Angular;ASP.NET Core ;Blazor;Node.js,.NET,.NET;Apache Kafka,npm,Docker;Kubernetes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36629,36630,,Bash/Shell;C;C++;Go;HTML/CSS;Java;JavaScript;J...,Bash/Shell;Go;HTML/CSS;Java;JavaScript;Kotlin;...,SQLite,MongoDB;PostgreSQL;SQLite,Heroku,Heroku,jQuery;Node.js;React.js,,GTK;NumPy;Qt,,npm,npm
36630,36631,Less than 1 year,C#;HTML/CSS;JavaScript;SQL,Bash/Shell;C#;HTML/CSS;JavaScript;Python;SQL,Microsoft SQL Server;MongoDB,Microsoft SQL Server;MongoDB,Heroku;Microsoft Azure,Heroku;Microsoft Azure,ASP.NET;ASP.NET Core ;Blazor;Express;jQuery;No...,Angular;Angular.js;ASP.NET Core ;Express;jQuer...,.NET,.NET,Docker;npm;Unity 3D,Docker;npm;Unity 3D;Unreal Engine
36631,36632,,HTML/CSS;JavaScript;Python;R;SQL;TypeScript,Go;Python;Rust;SQL;TypeScript,MongoDB;MySQL;PostgreSQL,MongoDB;PostgreSQL,AWS,Google Cloud,Express;Node.js;React.js,Express;Next.js;Node.js;React.js,NumPy;Scikit-learn;TensorFlow,.NET;NumPy;Scikit-learn;TensorFlow,npm,Docker;Kubernetes;npm
36632,36633,17,HTML/CSS;JavaScript;Ruby;SQL,HTML/CSS;JavaScript;Ruby;Rust,MySQL;PostgreSQL;Redis;SQLite,MySQL;PostgreSQL;Redis;SQLite,AWS;DigitalOcean,AWS;DigitalOcean,jQuery;Ruby on Rails,jQuery;Ruby on Rails,,,Docker,Docker;Kubernetes


In [4]:
# Join all "HaveWorked" and "WantToWorkWith" columns
df["Skills-Have"] = df["LanguageHaveWorkedWith"] + ";" + df["DatabaseHaveWorkedWith"]+ ";" + df["PlatformHaveWorkedWith"] + ";" + df["WebframeHaveWorkedWith"] + ";" + df["MiscTechHaveWorkedWith"] + ";" + df["ToolsTechHaveWorkedWith"]
df["Skills-Want"] = df["LanguageWantToWorkWith"] + ";" + df["DatabaseWantToWorkWith"]+ ";" + df["PlatformWantToWorkWith"] + ";" + df["WebframeWantToWorkWith"] + ";" + df["MiscTechWantToWorkWith"] + ";" + df["ToolsTechWantToWorkWith"]

# Remove original columns
df = df.drop(columns=["LanguageHaveWorkedWith","LanguageWantToWorkWith","DatabaseHaveWorkedWith","DatabaseWantToWorkWith","PlatformHaveWorkedWith","PlatformWantToWorkWith","WebframeHaveWorkedWith","WebframeWantToWorkWith","MiscTechHaveWorkedWith","MiscTechWantToWorkWith","ToolsTechHaveWorkedWith","ToolsTechWantToWorkWith"])
df

,ResponseId,YearsCodePro,Skills-Have,Skills-Want
0,1,,;;;;;,;;;;;
1,2,,JavaScript;TypeScript;;;;;,Rust;TypeScript;;;;;
2,3,5,C#;C++;HTML/CSS;JavaScript;Python;Microsoft SQ...,C#;C++;HTML/CSS;JavaScript;TypeScript;Microsof...
3,4,17,C#;JavaScript;SQL;TypeScript;Microsoft SQL Ser...,C#;SQL;TypeScript;Microsoft SQL Server;;ASP.NE...
4,5,3,C#;HTML/CSS;JavaScript;SQL;Swift;TypeScript;Cl...,C#;Elixir;F#;Go;JavaScript;Rust;TypeScript;Clo...
...,...,...,...,...
36629,36630,,Bash/Shell;C;C++;Go;HTML/CSS;Java;JavaScript;J...,Bash/Shell;Go;HTML/CSS;Java;JavaScript;Kotlin;...
36630,36631,Less than 1 year,C#;HTML/CSS;JavaScript;SQL;Microsoft SQL Serve...,Bash/Shell;C#;HTML/CSS;JavaScript;Python;SQL;M...
36631,36632,,HTML/CSS;JavaScript;Python;R;SQL;TypeScript;Mo...,Go;Python;Rust;SQL;TypeScript;MongoDB;PostgreS...
36632,36633,17,HTML/CSS;JavaScript;Ruby;SQL;MySQL;PostgreSQL;...,HTML/CSS;JavaScript;Ruby;Rust;MySQL;PostgreSQL...


## Get all available Skills and Position Requirements

In [5]:

# Get list of skills - Source: Stack Overflow Survey questions
skills_df = pandas.read_csv(filepath_or_buffer="data/skill-list.csv", sep=",", encoding="latin1")

# Get only Programming Language skills for now
#skills_df = skills_df.loc[skills_df['type'].isin(['Programming Language','Database'])]

# List required skills for the position and level of seniority
requirements = ['Python','Microsoft SQL Server','Microsoft Azure','Apache Spark','Scikit-learn','TensorFlow','Docker']
level_requirement = 'Architect'#'Mid-level'

# Get only selected skills
skills_df = skills_df.loc[skills_df['skill'].isin(requirements)]

skills_df

,skill,type,synonym
31,Python,Programming Language,NaN
50,Microsoft SQL Server,Database,sql server
68,Microsoft Azure,Cloud Platform,azure
100,Apache Spark,Other Framework,spark
113,Scikit-learn,Other Framework,NaN
116,TensorFlow,Other Framework,NaN
123,Docker,Developer Tools,NaN


## Prepare Data to Model

In [6]:

# For each row in the dataset, evaluate if the skill is mentioned or not

# get only the list of skills that are being looked for - then it filters only the relevant skills

skills = skills_df['skill']
for skill in skills:
    df[skill] = df["Skills-Have"].apply(lambda x: 1 if skill in x.split(';') else 0)

# Check if Skill is in the Want list
skills = skills_df['skill']
for skill in skills:
    df[skill+"-Want"] = df["Skills-Want"].apply(lambda x: 1 if skill in x.split(';') else 0)

df['YearsCodePro'] = df['YearsCodePro'].replace(np.nan, 0)
df['YearsCodePro'] = df["YearsCodePro"].apply(lambda x: '0' if x == '' or x == 'Less than 1 year' else x)
df['YearsCodePro'] = df["YearsCodePro"].apply(lambda x: '51' if x == 'More than 50 years' else x)

# Set the level category depending on years of experience
def setLevel(x):
    if int(x) <2:
        return 'Entry-level'
    elif int(x) >= 2 and int(x) <5:
        return 'Mid-level'
    elif int(x) >= 5 and int(x) <10:
        return  'Senior'
    elif int(x) >10:
        return 'Architect'

def setLevelNumber(x):
    if int(x) <2:
        return 3
    elif int(x) >= 2 and int(x) <5:
        return 2
    elif int(x) >= 5 and int(x) <10:
        return  1
    elif int(x) >=10:
        return 0

#df['Level'] = df["YearsCodePro"].apply(setLevel)
df['Level'] = df["YearsCodePro"].apply(setLevelNumber)

#seniority_level = ['Entry-level', 'Mid-level', 'Senior', 'Architect']
#for level in seniority_level:
#    df[level] = df["Level"].apply(lambda x: 1 if x == level else 0)


#seniority_level.remove(level_requirement)

# Leave remove level columns that are not required
#df = df.drop(columns=seniority_level)
#df = df.drop(columns='Level')




### Try to change the logic to use 0,1,2,3 values to reflect better the distance



df


,ResponseId,YearsCodePro,Skills-Have,Skills-Want,Python,Microsoft SQL Server,Microsoft Azure,Apache Spark,Scikit-learn,TensorFlow,Docker,Python-Want,Microsoft SQL Server-Want,Microsoft Azure-Want,Apache Spark-Want,Scikit-learn-Want,TensorFlow-Want,Docker-Want,Level
0,1,0,;;;;;,;;;;;,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3
1,2,0,JavaScript;TypeScript;;;;;,Rust;TypeScript;;;;;,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3
2,3,5,C#;C++;HTML/CSS;JavaScript;Python;Microsoft SQ...,C#;C++;HTML/CSS;JavaScript;TypeScript;Microsof...,1,1,0,0,0,0,0,0,1,0,0,0,0,0,1
3,4,17,C#;JavaScript;SQL;TypeScript;Microsoft SQL Ser...,C#;SQL;TypeScript;Microsoft SQL Server;;ASP.NE...,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0
4,5,3,C#;HTML/CSS;JavaScript;SQL;Swift;TypeScript;Cl...,C#;Elixir;F#;Go;JavaScript;Rust;TypeScript;Clo...,0,1,1,0,0,0,0,0,0,1,0,0,0,1,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36629,36630,0,Bash/Shell;C;C++;Go;HTML/CSS;Java;JavaScript;J...,Bash/Shell;Go;HTML/CSS;Java;JavaScript;Kotlin;...,1,0,0,0,0,0,0,1,0,0,0,0,0,0,3
36630,36631,0,C#;HTML/CSS;JavaScript;SQL;Microsoft SQL Serve...,Bash/Shell;C#;HTML/CSS;JavaScript;Python;SQL;M...,0,1,1,0,0,0,1,1,1,1,0,0,0,1,3
36631,36632,0,HTML/CSS;JavaScript;Python;R;SQL;TypeScript;Mo...,Go;Python;Rust;SQL;TypeScript;MongoDB;PostgreS...,1,0,0,0,1,1,0,1,0,0,0,1,1,1,3
36632,36633,17,HTML/CSS;JavaScript;Ruby;SQL;MySQL;PostgreSQL;...,HTML/CSS;JavaScript;Ruby;Rust;MySQL;PostgreSQL...,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0


In [7]:
# Get a sample of the rows
df = df.sample(1000)
df

,ResponseId,YearsCodePro,Skills-Have,Skills-Want,Python,Microsoft SQL Server,Microsoft Azure,Apache Spark,Scikit-learn,TensorFlow,Docker,Python-Want,Microsoft SQL Server-Want,Microsoft Azure-Want,Apache Spark-Want,Scikit-learn-Want,TensorFlow-Want,Docker-Want,Level
30901,30902,9,Bash/Shell;C;C++;JavaScript;Rust;TypeScript;Po...,C;JavaScript;Rust;TypeScript;Elasticsearch;Pos...,0,0,0,0,0,0,1,0,0,0,0,0,0,1,1
8653,8654,6,Bash/Shell;HTML/CSS;JavaScript;Python;SQL;Dyna...,Bash/Shell;Go;HTML/CSS;JavaScript;Python;Ruby;...,1,0,0,0,0,0,1,1,0,0,0,0,0,1,1
26424,26425,0,Kotlin;Python;Swift;MongoDB;Microsoft Azure;;;...,Kotlin;Python;Swift;MongoDB;Microsoft Azure;;;...,1,0,1,0,0,0,0,1,0,1,0,0,0,1,3
6113,6114,17,JavaScript;;;;;,;;;;;,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1400,1401,1,C#;HTML/CSS;SQL;TypeScript;Microsoft SQL Serve...,C#;Go;;;ASP.NET Core ;.NET;Uno Platform;,0,1,0,0,0,0,0,0,0,0,0,0,0,0,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20724,20725,20,MATLAB;R;SQL;MariaDB;Microsoft SQL Server;Orac...,MATLAB;R;SQL;MariaDB;Microsoft SQL Server;Orac...,0,1,0,0,0,0,1,0,1,0,0,0,0,1,0
17170,17171,13,Clojure;Java;SQL;PostgreSQL;Redis;AWS;;Spring;...,Clojure;Dart;SQL;PostgreSQL;AWS;;;Docker,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0
29986,29987,0,Haskell;MongoDB;;ASP.NET;React.js;.NET;Docker,C;C#;HTML/CSS;Java;JavaScript;SQL;Microsoft SQ...,0,0,0,0,0,0,1,0,1,0,0,0,0,1,3
19563,19564,0,C;C++;HTML/CSS;PHP;MariaDB;MySQL;;jQuery;Larav...,JavaScript;Python;MongoDB;;Vue.js;;,0,0,0,0,0,0,0,1,0,0,0,0,0,0,3


## KMeans to clusterize Employees

In [8]:
# Group people into clusters
number_clusters = df.drop(columns=["ResponseId","YearsCodePro","Skills-Have","Skills-Want"]).drop_duplicates().count()[0]
number_clusters
model = KMeans(n_clusters=number_clusters, random_state=42).fit(df.drop(columns=["ResponseId","YearsCodePro","Skills-Have","Skills-Want"]))
clusters = model.labels_
center = model.cluster_centers_

df
df_clusters = df
df_clusters["cluster"] = clusters
df_clusters

,ResponseId,YearsCodePro,Skills-Have,Skills-Want,Python,Microsoft SQL Server,Microsoft Azure,Apache Spark,Scikit-learn,TensorFlow,Docker,Python-Want,Microsoft SQL Server-Want,Microsoft Azure-Want,Apache Spark-Want,Scikit-learn-Want,TensorFlow-Want,Docker-Want,Level,cluster
30901,30902,9,Bash/Shell;C;C++;JavaScript;Rust;TypeScript;Po...,C;JavaScript;Rust;TypeScript;Elasticsearch;Pos...,0,0,0,0,0,0,1,0,0,0,0,0,0,1,1,4
8653,8654,6,Bash/Shell;HTML/CSS;JavaScript;Python;SQL;Dyna...,Bash/Shell;Go;HTML/CSS;JavaScript;Python;Ruby;...,1,0,0,0,0,0,1,1,0,0,0,0,0,1,1,1
26424,26425,0,Kotlin;Python;Swift;MongoDB;Microsoft Azure;;;...,Kotlin;Python;Swift;MongoDB;Microsoft Azure;;;...,1,0,1,0,0,0,0,1,0,1,0,0,0,1,3,84
6113,6114,17,JavaScript;;;;;,;;;;;,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,5
1400,1401,1,C#;HTML/CSS;SQL;TypeScript;Microsoft SQL Serve...,C#;Go;;;ASP.NET Core ;.NET;Uno Platform;,0,1,0,0,0,0,0,0,0,0,0,0,0,0,3,48
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20724,20725,20,MATLAB;R;SQL;MariaDB;Microsoft SQL Server;Orac...,MATLAB;R;SQL;MariaDB;Microsoft SQL Server;Orac...,0,1,0,0,0,0,1,0,1,0,0,0,0,1,0,38
17170,17171,13,Clojure;Java;SQL;PostgreSQL;Redis;AWS;;Spring;...,Clojure;Dart;SQL;PostgreSQL;AWS;;;Docker,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,27
29986,29987,0,Haskell;MongoDB;;ASP.NET;React.js;.NET;Docker,C;C#;HTML/CSS;Java;JavaScript;SQL;Microsoft SQ...,0,0,0,0,0,0,1,0,1,0,0,0,0,1,3,356
19563,19564,0,C;C++;HTML/CSS;PHP;MariaDB;MySQL;;jQuery;Larav...,JavaScript;Python;MongoDB;;Vue.js;;,0,0,0,0,0,0,0,1,0,0,0,0,0,0,3,24


In [9]:
#print(df_clusters.drop(columns=["ResponseId","YearsCodePro","Skills-Have","Skills-Want"]).drop_duplicates().count())
#print(df_clusters['cluster'].max())

employees_df = df_clusters.drop(columns=["YearsCodePro","Skills-Have","Skills-Want"]).sort_values("cluster")
employees_df


,ResponseId,Python,Microsoft SQL Server,Microsoft Azure,Apache Spark,Scikit-learn,TensorFlow,Docker,Python-Want,Microsoft SQL Server-Want,Microsoft Azure-Want,Apache Spark-Want,Scikit-learn-Want,TensorFlow-Want,Docker-Want,Level,cluster
26716,26717,1,0,0,0,0,0,0,1,0,0,0,0,0,0,3,0
30895,30896,1,0,0,0,0,0,0,1,0,0,0,0,0,0,3,0
29509,29510,1,0,0,0,0,0,0,1,0,0,0,0,0,0,3,0
33051,33052,1,0,0,0,0,0,0,1,0,0,0,0,0,0,3,0
6881,6882,1,0,0,0,0,0,0,1,0,0,0,0,0,0,3,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23498,23499,0,1,0,0,0,0,1,0,1,0,0,0,0,1,1,373
21079,21080,0,0,0,0,0,0,0,0,0,0,1,0,0,1,3,374
28847,28848,1,0,0,1,0,0,1,1,0,0,1,0,1,1,1,375
25656,25657,1,0,1,0,0,0,0,1,0,1,0,1,0,1,3,376


## KNN to find the Best Matches

In [10]:
# Find the one employee cluster closer to the requirement
knn = KNeighborsClassifier(n_neighbors=1)

knn.fit(employees_df.drop(columns=["ResponseId","cluster"]), employees_df['cluster'])

#size = len(requirements)*2
#size
position = np.ones(shape=(1, 1+len(requirements)*2))

if level_requirement == 'Entry-level':
    level_requirement = 3
elif level_requirement == 'Mid-level':
    level_requirement = 2
elif level_requirement == 'Senior':
    level_requirement = 1
elif level_requirement == 'Architect':
    level_requirement = 0

position[0][len(requirements)*2] = level_requirement

prediction = knn.predict(position)

prediction

array([73], dtype=int32)

## Get list of the matched Employees

In [11]:
# Employee found in the match
employees_df.loc[employees_df['cluster'] == prediction[0]]


,ResponseId,Python,Microsoft SQL Server,Microsoft Azure,Apache Spark,Scikit-learn,TensorFlow,Docker,Python-Want,Microsoft SQL Server-Want,Microsoft Azure-Want,Apache Spark-Want,Scikit-learn-Want,TensorFlow-Want,Docker-Want,Level,cluster
30715,30716,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,73


In [12]:
# Confirm if there is any perfect match
employees_df.loc[employees_df['Python'] == 1].loc[employees_df['Microsoft SQL Server'] == 1].loc[employees_df['Microsoft Azure'] == 1].loc[employees_df['Apache Spark'] == 1].loc[employees_df['Scikit-learn'] == 1].loc[employees_df['TensorFlow'] == 1].loc[employees_df['Docker'] == 1]


,ResponseId,Python,Microsoft SQL Server,Microsoft Azure,Apache Spark,Scikit-learn,TensorFlow,Docker,Python-Want,Microsoft SQL Server-Want,Microsoft Azure-Want,Apache Spark-Want,Scikit-learn-Want,TensorFlow-Want,Docker-Want,Level,cluster
30715,30716,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,73
17816,17817,1,1,1,1,1,1,1,0,0,0,0,0,0,0,0,85


In [25]:
# Check information for each employee considered
import pandas
df_original = pandas.read_csv(filepath_or_buffer="data/survey_results_public_1.csv", sep=",", encoding="latin1")
df_original = df_original[["ResponseId","YearsCodePro","LanguageHaveWorkedWith","LanguageWantToWorkWith","DatabaseHaveWorkedWith","DatabaseWantToWorkWith","PlatformHaveWorkedWith","PlatformWantToWorkWith","WebframeHaveWorkedWith","WebframeWantToWorkWith","MiscTechHaveWorkedWith","MiscTechWantToWorkWith","ToolsTechHaveWorkedWith","ToolsTechWantToWorkWith"]]

df_original.loc[df_original['ResponseId'] == 31107]


,ResponseId,YearsCodePro,LanguageHaveWorkedWith,LanguageWantToWorkWith,DatabaseHaveWorkedWith,DatabaseWantToWorkWith,PlatformHaveWorkedWith,PlatformWantToWorkWith,WebframeHaveWorkedWith,WebframeWantToWorkWith,MiscTechHaveWorkedWith,MiscTechWantToWorkWith,ToolsTechHaveWorkedWith,ToolsTechWantToWorkWith
31106,31107,3,Assembly;Bash/Shell;C;Haskell;HTML/CSS;Java;Ja...,HTML/CSS;JavaScript;Python;Scala;SQL,Microsoft SQL Server;MongoDB;MySQL;PostgreSQL;...,MySQL;PostgreSQL;SQLite,AWS;Firebase;Microsoft Azure;OVH,AWS;Firebase;Google Cloud;Microsoft Azure;OVH,Angular;Django;jQuery;Node.js;React.js,Django,Apache Spark;Hadoop;Keras;NumPy;Pandas;Scikit-...,Apache Spark;Hadoop;Keras;NumPy;Pandas;Scikit-...,Docker,Docker;Kubernetes;Terraform
